<a href="https://colab.research.google.com/github/vifirsanova/llm-dmt/blob/main/get_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install -U angle-emb

In [ ]:
import torch
from angle_emb import AnglE, Prompts
from angle_emb.utils import cosine_similarity

angle = AnglE.from_pretrained('NousResearch/Llama-2-7b-hf',
                              pretrained_lora_path='SeanLee97/angle-llama-7b-nli-v2',
                              pooling_strategy='last',
                              is_llm=True,
                              torch_dtype=torch.float16).cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/320M [00:00<?, ?B/s]

In [ ]:
Prompts.list_prompts()

doc_vec = angle.encode([
    {'text': 'This'},
    {'text': 'is'},
    {'text': 'a'},
    {'text': 'sample'},
    {'text': 'sentence'}
], prompt=Prompts.C)

doc_vec

Prompts.A = 'Summarize sentence "{text}" in one word:"'
Prompts.B = 'You can only output one word. Summarize "{text}":"'
Prompts.C = 'Represent this sentence for searching relevant passages: {text}'


array([[ 0.07928467,  0.2310791 ,  1.5292969 , ...,  0.4934082 ,
        -1.0556641 ,  0.46728516],
       [-0.37548828, -1.8476562 ,  0.48779297, ...,  0.00518036,
         0.48828125,  0.04760742],
       [-0.37109375, -0.63427734, -0.1274414 , ...,  0.3017578 ,
         0.6582031 ,  0.32055664],
       [-2.4082031 ,  0.76904297, -0.41577148, ...,  0.08520508,
         0.6845703 ,  1.4423828 ],
       [-3.3710938 , -0.13769531,  2.3515625 , ...,  0.7133789 ,
        -0.40893555, -0.08282471]], dtype=float32)

In [ ]:
labels = ['This', 'is', 'a', 'sample', 'sentence']

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, perplexity=3, random_state=42)
embeddings_2d = tsne.fit_transform(doc_vec)

In [ ]:
import networkx as nx

# Create a directed graph
G = nx.DiGraph()

# Add nodes to the graph
for i, label in enumerate(labels):
    G.add_node(i, label=label, pos=(embeddings_2d[i, 0], embeddings_2d[i, 1]))

# Add edges to the graph (example: create a simple sequence)
for i in range(len(labels) - 1):
    G.add_edge(i, i + 1)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Define the sentence
sentence = "This is a sample sentence"

# Tokenize the sentence
tokens = sentence.split()

# Define the c-structure
c_structure = [
    ('S', 'NP', 0),
    ('NP', 'Det', 0),
    ('Det', 'this', 0),
    ('NP', 'N', 1),
    ('N', 'is', 1),
    ('S', 'VP', 1),
    ('VP', 'V', 1),
    ('V', 'is', 1),
    ('VP', 'NP', 2),
    ('NP', 'Det', 2),
    ('Det', 'a', 2),
    ('NP', 'N', 3),
    ('N', 'sample', 3),
    ('VP', 'N', 4),
    ('N', 'sentence', 4)
]

# Define the f-structure
f_structure = {
    'SUBJ': 'this',
    'PRED': 'is',
    'OBJ': 'a sample sentence'
}

# Create a graph for c-structure
c_graph = nx.DiGraph()
c_graph.add_node('S')

# Add nodes and edges for c-structure
for (parent, child, index) in c_structure:
    c_graph.add_node(child, label=child)
    c_graph.add_edge(parent, child, label=tokens[index])

# Create a graph for f-structure
f_graph = nx.DiGraph()
f_graph.add_node('F-Structure')

# Add nodes and edges for f-structure
for attr, value in f_structure.items():
    f_graph.add_node(attr, label=attr)
    f_graph.add_edge('F-Structure', attr, label=attr)
    f_graph.add_node(value, label=value)
    f_graph.add_edge(attr, value, label=value)

In [ ]:
# Calculate the common nodes
common_nodes = set(c_graph.nodes) & set(G.nodes)
print("Common Nodes:", common_nodes)

# Calculate the common edges
common_edges = set(c_graph.edges) & set(G.edges)
print("Common Edges:", common_edges)

# Create a new graph for the intersection
G_intersect = nx.DiGraph()

# Add the common nodes
G_intersect.add_nodes_from(common_nodes)

# Add the common edges
G_intersect.add_edges_from(common_edges)

# Print the intersection graph
print("Intersection Graph Nodes:", G_intersect.nodes)
print("Intersection Graph Edges:", G_intersect.edges)

Common Nodes: set()
Common Edges: set()
Intersection Graph Nodes: []
Intersection Graph Edges: []


In [ ]:
# Calculate the common nodes
common_nodes = set(f_graph.nodes) & set(G.nodes)
print("Common Nodes:", common_nodes)

# Calculate the common edges
common_edges = set(f_graph.edges) & set(G.edges)
print("Common Edges:", common_edges)

# Create a new graph for the intersection
G_intersect = nx.DiGraph()

# Add the common nodes
G_intersect.add_nodes_from(common_nodes)

# Add the common edges
G_intersect.add_edges_from(common_edges)

# Print the intersection graph
print("Intersection Graph Nodes:", G_intersect.nodes)
print("Intersection Graph Edges:", G_intersect.edges)

Common Nodes: set()
Common Edges: set()
Intersection Graph Nodes: []
Intersection Graph Edges: []


In [ ]:
# Calculate the common nodes
common_nodes = set(f_graph.nodes) & set(c_graph.nodes)
print("Common Nodes:", common_nodes)

# Calculate the common edges
common_edges = set(f_graph.edges) & set(c_graph.edges)
print("Common Edges:", common_edges)

# Create a new graph for the intersection
G_intersect = nx.DiGraph()

# Add the common nodes
G_intersect.add_nodes_from(common_nodes)

# Add the common edges
G_intersect.add_edges_from(common_edges)

# Print the intersection graph
print("Intersection Graph Nodes:", G_intersect.nodes)
print("Intersection Graph Edges:", G_intersect.edges)

Common Nodes: {'is', 'this'}
Common Edges: set()
Intersection Graph Nodes: ['is', 'this']
Intersection Graph Edges: []
